In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = (
    SparkSession.builder.appName("iot").getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")
spark.sparkContext.version


'4.0.0'